In [5]:
!pip install azure-storage-blob


In [7]:
from azure.storage.blob import BlobServiceClient
import os


connection_string = "DefaultEndpointsProtocol=https;AccountName=nflbigdatabowl2023;AccountKey=dgaG9SIdALFWMlB3vs7ogNx+pN/l8dH7XCSFWp2WdNGn/915zmXoDDeCB0t7CR+LWZe8FJBdV70Q+AStY3KHzA==;EndpointSuffix=core.windows.net"


container_name = "raw"


blob_service = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service.get_container_client(container_name)


try:
    container_client.create_container()
    print(f"Container '{container_name}' created.")
except Exception as e:
    print(f"Container might already exist or another issue: {e}")


Container might already exist or another issue: The specified container already exists.
RequestId:f20d205a-801e-0055-1a6f-d5dcde000000
Time:2025-06-04T16:42:20.8550941Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:f20d205a-801e-0055-1a6f-d5dcde000000
Time:2025-06-04T16:42:20.8550941Z</Message></Error>


In [4]:
from azure.storage.blob import BlobServiceClient
import os
import glob

# Azure connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=nflbigdatabowl2023;AccountKey=dgaG9SIdALFWMlB3vs7ogNx+pN/l8dH7XCSFWp2WdNGn/915zmXoDDeCB0t7CR+LWZe8FJBdV70Q+AStY3KHzA==;EndpointSuffix=core.windows.net"
container_name = "raw"

# Connect to blob service and container
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Local folder path
folder_path = r"C:\Users\USER\Downloads\nfl-big-data-bowl-2023"

# Get list of all CSV and Excel files
file_patterns = ["*.csv", "*.xlsx"]
files_to_upload = []
for pattern in file_patterns:
    files_to_upload.extend(glob.glob(os.path.join(folder_path, pattern)))

# Upload each file
for file_path in files_to_upload:
    try:
        file_name = os.path.basename(file_path)
        with open(file_path, "rb") as data:
            container_client.upload_blob(name=file_name, data=data, overwrite=True)
        print(f"✅ Uploaded: {file_name}")
    except Exception as e:
        print(f"❌ Failed to upload {file_name}: {e}")


✅ Uploaded: games.csv
✅ Uploaded: pffScoutingData.csv
✅ Uploaded: players.csv
✅ Uploaded: plays.csv
✅ Uploaded: week1.csv
✅ Uploaded: week2.csv
✅ Uploaded: week3.csv
✅ Uploaded: week4.csv
✅ Uploaded: week5.csv
✅ Uploaded: week6.csv
✅ Uploaded: week7.csv
✅ Uploaded: week8.csv


In [6]:
from azure.storage.blob import BlobServiceClient

# Azure connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=nflbigdatabowl2023;AccountKey=dgaG9SIdALFWMlB3vs7ogNx+pN/l8dH7XCSFWp2WdNGn/915zmXoDDeCB0t7CR+LWZe8FJBdV70Q+AStY3KHzA==;EndpointSuffix=core.windows.net"
container_name = "raw"

# Connect to blob service and container
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# List blobs in the container
print(f"📁 Files in container '{container_name}':")
try:
    blobs = container_client.list_blobs()
    found = False
    for blob in blobs:
        print(f"  - {blob.name}")
        found = True
    if not found:
        print("  (No files found)")
except Exception as e:
    print(f"❌ Error listing blobs: {e}")


📁 Files in container 'raw':
  - games.csv
  - pffScoutingData.csv
  - players.csv
  - plays.csv
  - week1.csv
  - week2.csv
  - week3.csv
  - week4.csv
  - week5.csv
  - week6.csv
  - week7.csv
  - week8.csv


In [24]:
# one-cell script: set env var, download CSV to Pandas

import os
import io
import pandas as pd
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceNotFoundError

# 1️Set the environment variable for this session only
os.environ["AZURE_STORAGE_CONNECTION_STRING"] = (
    "DefaultEndpointsProtocol=https;"
    "AccountName=nflbigdatabowl2023;"
    "AccountKey=dgaG9SIdALFWMlB3vs7ogNx+pN/l8dH7XCSFWp2WdNGn/915zmXoDDeCB0t7CR+LWZe8FJBdV70Q+AStY3KHzA==;"
    "EndpointSuffix=core.windows.net"
)

# 2️⃣  Read the connection string back out
conn_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

# 3️⃣  Blob details
container_name = "raw"
blob_name = "plays.csv"

try:
    # 4️⃣  Connect and download
    blob_service = BlobServiceClient.from_connection_string(conn_str)
    blob_client = blob_service.get_blob_client(container=container_name, blob=blob_name)

    download_stream = blob_client.download_blob()
    byte_data = download_stream.readall()      # OK for ~hundreds of MB

    # 5️⃣  Load into Pandas
    df = pd.read_csv(io.BytesIO(byte_data))

    print("✅ CSV loaded. Shape:", df.shape)
    display(df.head())  # Jupyter-friendly; or print(df.head()) in plain Python

except ResourceNotFoundError:
    print(f"❌ Blob '{blob_name}' not found in container '{container_name}'.")
except Exception as e:
    print(f"❌ Download failed: {e}")


✅ CSV loaded. Shape: (8557, 32)


,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone


In [28]:
# -------------------------------------------------------------
#  NFL Big Data Bowl 2023  ·  FULL PRE-PROCESSING PIPELINE
# -------------------------------------------------------------
#  Requires:  pip install azure-storage-blob pandas pyarrow
# -------------------------------------------------------------
import os, io, re
import pandas as pd
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceNotFoundError

# -- 0.  Get connection string from env -------------------------------------------------
conn_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
if not conn_str:
    raise RuntimeError("Set AZURE_STORAGE_CONNECTION_STRING first (see Option A).")

# -- 1.  Download plays.csv from the raw container --------------------------------------
container_name, blob_name = "raw", "plays.csv"
try:
    blob = (
        BlobServiceClient.from_connection_string(conn_str)
        .get_blob_client(container=container_name, blob=blob_name)
    )
    byte_data = blob.download_blob().readall()
    df = pd.read_csv(io.BytesIO(byte_data))
    print("✅ Loaded:", df.shape, "columns:", list(df.columns)[:8], "…")
except ResourceNotFoundError:
    raise FileNotFoundError(f"{blob_name} not found in {container_name}")
except Exception as e:
    raise RuntimeError("Download failed:", e)

# ---------- 2. COLUMN-TYPE ENFORCEMENT -------------------------------------------------
for col in ["gameId", "playId"]:
    if col in df.columns:
        df[col] = df[col].astype("int64", errors="ignore")

# ---------- 3. NULL HANDLING -----------------------------------------------------------
# Use columns that really exist in plays.csv
critical_cols = [c for c in ["possessionTeam", "playDescription"] if c in df.columns]
if critical_cols:
    df = df.dropna(subset=critical_cols)
    print(f"Dropped rows with nulls in {critical_cols}; rows now:", len(df))

# ---------- 4. FEATURE ENGINEERING -----------------------------------------------------
if {"absoluteYardlineNumber", "yardlineNumber"}.issubset(df.columns):
    df["yardsToGo_calc"] = (
        df["absoluteYardlineNumber"] - df["yardlineNumber"]
    )

    # Merge with existing yardsToGo if present
    if "yardsToGo" in df.columns:
        df["yardsToGo"] = df["yardsToGo"].fillna(df["yardsToGo_calc"])
    else:
        df.rename(columns={"yardsToGo_calc": "yardsToGo"}, inplace=True)

# Simple pass/run flag derived from playDescription
if "playDescription" in df.columns:
    df["is_pass"] = df["playDescription"].str.contains(r"\bpass\b", flags=re.I).astype("int8")

# ---------- 5. OUTLIER FILTERING -------------------------------------------------------
if "yardsToGo" in df.columns:
    before = len(df)
    df = df.query("yardsToGo >= 0 and yardsToGo <= 100")
    print("Filtered impossible yardsToGo:", before - len(df), "rows removed")

# ---------- 6. DEDUPLICATION -----------------------------------------------------------
if {"gameId", "playId"}.issubset(df.columns):
    df = df.drop_duplicates(subset=["gameId", "playId"])

# ---------- 7. EXPORT CLEAN DATA AS PARQUET -------------------------------------------
out_file = "plays_clean.parquet"
df.to_parquet(out_file, index=False, compression="snappy")
print("💾 Saved cleaned Parquet:", out_file, "final shape:", df.shape)

# ---------- 8. OPTIONAL – Upload back to a 'curated' container -------------------------
try:
    curated_client = (
        BlobServiceClient.from_connection_string(conn_str)
        .get_container_client("curated")
    )
    curated_client.create_container()  # safe: ignored if exists
    with open(out_file, "rb") as fh:
        curated_client.upload_blob("plays_clean.parquet", fh, overwrite=True)
    print("🚀 Uploaded to curated/plays_clean.parquet")
except Exception as e:
    print("Upload skipped/failed:", e)


✅ Loaded: (8557, 32) columns: ['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam'] …
Dropped rows with nulls in ['possessionTeam', 'playDescription']; rows now: 8557
Filtered impossible yardsToGo: 0 rows removed
💾 Saved cleaned Parquet: plays_clean.parquet final shape: (8557, 34)
🚀 Uploaded to curated/plays_clean.parquet


In [32]:
import pandas as pd, io, os, pyarrow.parquet as pq
from azure.storage.blob import BlobServiceClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import OneHotEncoder
from sklearn.compose         import ColumnTransformer
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression
from sklearn.metrics         import roc_auc_score
import numpy as np

# 1. Load Parquet from curated container
conn = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
blob = (BlobServiceClient.from_connection_string(conn)
        .get_blob_client("curated", "plays_clean.parquet"))
df   = pq.read_table(io.BytesIO(blob.download_blob().readall())).to_pandas()

# 2. Derive field_zone if it isn't there
if "field_zone" not in df.columns:
    def derive_field_zone(row):
        y = row["yardlineNumber"]
        if   1  <= y <= 20:  return "Red Zone"
        elif 21 <= y <= 50:  return "Midfield"
        else:                return "Own Territory"
    df["field_zone"] = df.apply(derive_field_zone, axis=1)

# 3. Define target & features
y = df["is_pass"]
X = df[["down", "yardsToGo", "quarter", "field_zone", "possessionTeam"]]

# 4. Build preprocessing + model pipeline
categorical = ["field_zone", "possessionTeam"]
numeric     = ["down", "yardsToGo", "quarter"]

pre = ColumnTransformer(
        [("cat", OneHotEncoder(handle_unknown="ignore"), categorical)],
        remainder="passthrough")

model = Pipeline(steps=[
        ("pre", pre),
        ("clf", LogisticRegression(max_iter=1000, n_jobs=-1))
])

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
print("ROC-AUC:", round(auc, 3))

# Optional: show top coefficients
names = model.named_steps["pre"].get_feature_names_out()
coefs = model.named_steps["clf"].coef_[0]
top   = pd.Series(coefs, index=names).sort_values(key=np.abs, ascending=False)[:10]
print("\nTop drivers of pass likelihood:\n", top)


ROC-AUC: 0.624

Top drivers of pass likelihood:
 cat__possessionTeam_LA     0.911016
cat__possessionTeam_CHI   -0.822047
cat__possessionTeam_TB     0.727057
cat__possessionTeam_MIN    0.718361
cat__possessionTeam_BAL   -0.702872
cat__possessionTeam_ATL    0.625297
cat__possessionTeam_PIT    0.570056
cat__possessionTeam_PHI   -0.540738
cat__possessionTeam_CLE   -0.465204
cat__possessionTeam_NO    -0.441274
dtype: float64
